In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 96.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 26.9 MB/s et

In [ ]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
train_set = pd.read_csv("/content/gdrive/MyDrive/TamilNLP/train_ta_hi_ta_4480.csv",header=None,sep=',')
validation_set = pd.read_csv("/content/gdrive/MyDrive/TamilNLP/Org_Dataset/ta-misogyny-dev.csv",header=None,sep='\t')
test_set = pd.read_csv("/content/gdrive/MyDrive/TamilNLP/Org_Dataset/tamil_test_with_labels.csv", header=None)
print(f"Training data : {train_set.shape} \nValidation data : {validation_set.shape} \nTest data : {test_set.shape}")

Training data : (4480, 2) 
Validation data : (560, 2) 
Test data : (699, 2)


In [ ]:
def modify_dataset(df):
  df.rename(columns={0:'Label', 1:'Text'}, inplace=True)
  df = df[['Text', 'Label']]
  return df
train_set = modify_dataset(train_set)
validation_set = modify_dataset(validation_set)
test_set = modify_dataset(test_set)

In [ ]:
num_labels = len(train_set['Label'].unique())
keys = list(train_set['Label'].unique())
values = list(range(0, num_labels))
label_dict = dict(zip(keys,values))
train_set['Label'] = train_set['Label'].apply(lambda x:label_dict[x])
test_set['Label'] = test_set['Label'].apply(lambda x:label_dict[x])
validation_set['Label'] = validation_set['Label'].apply(lambda x:label_dict[x])
label_dict

{'Hope-Speech': 0,
 'None-of-the-above': 1,
 'Homophobia': 2,
 'Misandry': 3,
 'Counter-speech': 4,
 'Misogyny': 5,
 'Xenophobia': 6,
 'Transphobic': 7,
 'Not-Tamil': 8}

In [ ]:
def oversample(df):
    classes = df['Label'].value_counts().to_dict()
    most = max(classes.values())
    classes_list = []
    for key in classes:
        classes_list.append(df[df['Label'] == key])
    classes_sample = []
    for i in range(1,len(classes_list)):
        classes_sample.append(classes_list[i].sample(most, replace=True))
    df_maybe = pd.concat(classes_sample)
    final_df = pd.concat([df_maybe,classes_list[0]], axis=0)
    final_df = final_df.reset_index(drop=True)
    return pd.DataFrame({'Text': final_df['Text'].tolist(), 'Label': final_df['Label'].tolist()})
training_set = oversample(train_set)
training_set = training_set.sample(frac=1).reset_index(drop=True)

In [ ]:
epochs=3
model_args = ClassificationArgs()
model_args.overwrite_output_dir=True
model_args.eval_batch_size=8
model_args.train_batch_size=32
model_args.learning_rate=4e-5
model_args.save_model_every_epoch = True

In [ ]:
model = ClassificationModel(
    'bert',
    'google/muril-base-cased',
    num_labels=9,
    args=model_args,
    use_cuda=True,
    tokenizer_type="bert",
    tokenizer_name='google/muril-base-cased'
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_losses = []
train_accuracies = []

for i in range(epochs):
    model.train_model(training_set,eval_data=validation_set,acc=sklearn.metrics.classification_report)
    result, model_outputs, preds_list = model.eval_model(test_set,acc=sklearn.metrics.classification_report)
    #print(preds_list, model_outputs)
    for j in result.values():
        print(j)


/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/23328 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/729 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/699 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/88 [00:00<?, ?it/s]

0.4252016557124808
              precision    recall  f1-score   support

         0.0       0.33      0.54      0.41        26
         1.0       0.77      0.76      0.77       416
         2.0       0.62      0.62      0.62         8
         3.0       0.57      0.63      0.60       127
         4.0       0.33      0.34      0.33        47
         5.0       0.53      0.38      0.44        48
         6.0       0.46      0.24      0.32        25
         7.0       0.00      0.00      0.00         2

    accuracy                           0.65       699
   macro avg       0.45      0.44      0.44       699
weighted avg       0.66      0.65      0.65       699

1.6087292026389728


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
